<a href="https://colab.research.google.com/github/A-Burnhard/Major-Atmospheric-Gamma-Imaging-Cherenkov-Telescope-project-MAGIC-/blob/main/MAGIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler


**Dataset** 
 1.Title of Database: MAGIC gamma telescope data 2004

2. Sources:

   (a) Original owner of the database:

       R. K. Bock
       Major Atmospheric Gamma Imaging Cherenkov Telescope project (MAGIC)
       http://wwwmagic.mppmu.mpg.de
       rkb@mail.cern.ch


In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive//Machine Learning/magic04.data'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class" ]
df = pd.read_csv(file_path, names=cols)
df.head()


In [ ]:
df["class"] = (df["class"] == "g").astype(int)

df.head()

In [ ]:
for label in cols[:-1]:
  data_gamma= df[df["class"]==1][label],
  data_hadron = df[df["class"]==0][label], 
 
  plt.hist(data_gamma, color='blue',label='gamma', alpha=0.7, density=True, histtype = 'bar')
  plt.hist(data_hadron, color='red', label='hadron',alpha=0.7, density=True, histtype = 'bar')
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

**Train, Validation, test datasets**

In [ ]:
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])


Between 60% and 80% wil go towards validation and from 80% to 100% will be the test data.

**Scaling dataset** using Standard Deviation
Hstack = convert to 2D Numpy

In [ ]:
def scale_dataset(dataframe, oversample):
  x = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler= StandardScaler()
  x = scaler.fit_transform(x)

  if oversample:
    ros = RandomOverSampler()
    x,y = ros.fit_resample(x,y)

  data = np.hstack((x,np.reshape(y,(-1,1))))

  return data, x, y

In [ ]:
train, x_train, y_train = scale_dataset(train, oversample=True)
valid, x_valid, y_valid = scale_dataset(valid, oversample=False)
test, x_test, y_test = scale_dataset(test, oversample=False)




**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_pred = knn_model.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred))

Naive Bayes 

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)

In [ ]:
y_pred = nb_model.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg_model = LogisticRegression()
lg_model = lg_model.fit(x_train, y_train)

In [ ]:
y_pred = lg_model.predict(x_test)
print(classification_report(y_test, y_pred))

**Support Vector Machine (SVM)**

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model = SVC()
svm_model = svm_model.fit(x_train,y_train)

In [ ]:
y_pred = svm_model.predict(x_test)
print(classification_report(y_test, y_pred))

**Neural Networks (NN)**

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode labels 'g' and 'h' as 0 and 1 respectively
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


In [ ]:
y_train = np.array(y_train).astype('float32')


In [ ]:
def plot_history(history):
  fig,(ax1,ax2) = plt.subplots(1,2, figsize= (10,4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Binary Crossentropy')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)

  plt.show()

In [ ]:
plot_history(history)

In [ ]:
def train_model(x_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(10,)),
        tf.keras.layers.Dropout(dropout_prob),
        tf.keras.layers.Dense(num_nodes, activation='relu'),
        tf.keras.layers.Dropout(dropout_prob),
        tf.keras.layers.Dense(1, activation='sigmoid'),
        ])

  nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), 
                 loss='binary_crossentropy', 
                 metrics=['accuracy'])
  
  history =  nn_model.fit(
      x_train, y_train, epochs=epochs, batch_size = batch_size, validation_split=0.2, verbose=0
  )
  return nn_model, history
 

In [ ]:
least_val_loss = float('inf')
least_loss_model = None
epochs = 100
for num_nodes in [16,32,64]:
  for dropout_prob in [0, 0.2]:
    for lr in [0.1,0.005, 0.001]:
      for batch_size in [32, 64, 120]: 
        model , history = train_model(x_train, y_train,num_nodes, dropout_prob, lr, batch_size, epochs)
        plot_history(history)
        val_loss = model.evaluate(x_valid, y_valid)[0]
        if val_loss < least_val_loss:
          least_val_loss = val_loss
          least_loss_model = model


In [ ]:
y_pred = least_loss_model.predict(x_test)
y_pred = (y_pred >0.5).astype(int).reshape(-1,1)


In [ ]:
print(classification_report(y_test,y_pred))